This notebook calculates the carbon footprint linked to the Quebec economy as described in the IO table generated in technology_matrix.ipynb and to the final demand vectors created for each quintile of Quebec households.

In [1]:
import pandas as pd
import numpy as np
import pymrio

In [2]:
path_to_exiobase_unzipped_folder = 'my_own_path/exiobase3.4/IOT_2011_pxp/'
io = pymrio.parse_exiobase3(path_to_exiobase_unzipped_folder)

To calculate the carbon footprint we will need the normalized emissions of EXIOBASE and its sectors. Pymrio does not generate normalized emissions from parsing exiobase. We need to calculate them first:

In [3]:
io.calc_all()

These emissions are in M€. Since our scale is the household, we bring back emissions to €:

In [4]:
io.satellite.S.loc[
    [i for i in io.satellite.S.index if io.satellite.unit.loc[i,'unit'] not in ['M.EUR','1000 p','M.hr']]
                    ]/=1000000

In the same fashion as with the technology matrix, we copy the normalized emissions of Canada and rename them QC. 

Note: direct emissions from technologies are considered the same in Canada and Quebec as technologies themselves were not changed.

In [5]:
S = io.satellite.S.copy()
S = pd.concat([S,S.loc[:,'CA']],axis=1)
S.columns = pd.MultiIndex.from_product([io.get_regions().tolist()+['QC'],io.get_sectors()],names=('region', 'sector'))
S[('QC','grid_mix')] = pd.Series(0, S.index)

To get to the catbon footprint, we need a characterization matrix matching each GHG emission to its Global Warming Pontential:

such a characterization matrix, directly matched to EXIOBASE flows can be found here: https://zenodo.org/record/3890339

It uses the IMPACT World+ impact assessment methodology.

In [7]:
C = pd.read_csv('path_to_lonely_mountain/C_exio_IW.csv')
C = C.set_index('Unnamed: 0')
C = C.reindex(S.index,axis=1).fillna(0)

We got the emissions and impacts covered. Now let's load the technologies and households of Quebec that we produced before:

In [8]:
A = np.load('one_matrix_to_rule_them_all.npy')
index = pd.MultiIndex.from_product([io.get_regions().tolist()+['QC'],io.get_sectors()],names=['region','sector'])
index = index.tolist()+[('QC','grid_mix')]
A = pd.DataFrame(A, index=index, columns=index)

Y = pd.DataFrame(np.load('final_demand.npy'),index=A.index, columns=['1st_quintile','2nd_quintile','3rd_quintile','4th_quintile','5th_quintile'])

The determination of life cycle impacts follows the equation:

\begin{equation}
\textbf{impacts} =
\textbf{C} \cdot \textbf{S} \cdot(\textbf{I} - \textbf{A}) ^{-1} \cdot \textbf{y}
\end{equation}

or:

\begin{equation}
\textbf{impacts} =
\textbf{C} \cdot \textbf{S} \cdot \textbf{L}
\end{equation}

We already have S, y and A. We need to calculate L:

In [9]:
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
L = pd.DataFrame(np.linalg.solve(I-A,Y),A.index, Y.columns)

Indirect emissions of the households, i.e., emissions coming from the production of good/service bought are:

In [10]:
indirect_emissions = S.dot(L)

To these indirect emissions, direct emissions must be added, i.e., emissions coming from the use by households of a good/service, e.g., using one's car. In fact, the previous equation was incomplete:

\begin{equation}
\textbf{impacts} =
\textbf{C} \cdot (\textbf{S} \cdot(\textbf{I} - \textbf{A}) ^{-1} \cdot \textbf{y} + \textbf{SY})
\end{equation}

The direct emissions of the Canadian (matrix SY) will once again be adapted for Quebec. Again, we divide by 1,000,000 as the normalized emissions of the Canadian households are in M€.

In [11]:
direct_emissions = (pd.DataFrame(io.satellite.SY.loc[:, ('CA','Final consumption expenditure by households')])/1000000)
direct_emissions = pd.concat([direct_emissions]*5,axis=1)
direct_emissions.columns = Y.columns
direct_emissions *= Y.sum()

total emissions are defined as:

In [12]:
total_emissions = indirect_emissions + direct_emissions
total_emissions

,1st_quintile,2nd_quintile,3rd_quintile,4th_quintile,5th_quintile
Taxes less subsidies on products purchased: Total,8.377709e+02,1.229980e+03,1.797674e+03,2.287748e+03,3.309334e+03
Other net taxes on production,1.474382e+03,1.848995e+03,2.504323e+03,3.030058e+03,4.159734e+03
"Compensation of employees; wages, salaries, & employers social contributions: Low-skilled",7.732065e+02,1.151410e+03,1.648157e+03,2.096693e+03,3.069062e+03
"Compensation of employees; wages, salaries, & employers social contributions: Medium-skilled",5.038692e+03,7.492575e+03,1.079406e+04,1.374951e+04,2.027601e+04
"Compensation of employees; wages, salaries, & employers social contributions: High-skilled",3.634125e+03,5.351354e+03,7.887205e+03,9.842017e+03,1.466180e+04
Operating surplus: Consumption of fixed capital,4.377967e+03,6.081178e+03,8.508804e+03,1.045915e+04,1.480233e+04
Operating surplus: Rents on land,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Operating surplus: Royalties on resources,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Operating surplus: Remaining net operating surplus,7.121012e+03,9.710604e+03,1.341281e+04,1.649289e+04,2.307464e+04
Employment: Low-skilled male,6.537310e+01,9.535202e+01,1.381776e+02,1.777215e+02,2.525973e+02


Multiply with the characterization factors to get the carbon footprint:

In [13]:
impact_category = 'Climate change, short term (kg CO2 eq (short))'

GWP_per_quintile_per_household = pd.DataFrame(C.dot(total_emissions).loc[impact_category])

Note that sinc ethe begining we are talking about € because that is the currency used in EXIOBASE. We thus need to apply a convertion rate, for the year of reference. You can find convertion rates here (https://www.statista.com/statistics/412804/euro-to-canadian-dollar-average-annual-exchange-rate/). Our reference year is 2011, so:

In [14]:
GWP_per_quintile_per_household /= 1.38

Carbon footprint (in kgCO2eq) per household in Quebec is therefore:

In [15]:
GWP_per_quintile_per_household

,"Climate change, short term (kg CO2 eq (short))"
1st_quintile,13255.210581
2nd_quintile,20063.035735
3rd_quintile,27986.498012
4th_quintile,35856.978648
5th_quintile,51583.616755


We can also look at the carbon footprint per dollar spent:

In [16]:
GWP_per_quintile_per_dollar = GWP_per_quintile_per_household.T/Y.sum()
GWP_per_quintile_per_dollar.T

,"Climate change, short term (kg CO2 eq (short))"
1st_quintile,0.572737
2nd_quintile,0.613518
3rd_quintile,0.604094
4th_quintile,0.621669
5th_quintile,0.621702


Or if we scale up to all of Quebec (assuming 3,411,000 households https://www.stat.gouv.qc.ca/statistiques/conditions-vie-societe/depenses-avoirs-dettes/depenses/effectif.htm)

In [17]:
GWP_quebec_total = GWP_per_quintile_per_household.sum().sum()*3411000/5
GWP_quebec_total

101474070764.91489

The calculated carbon footprint of Quebec in 2011 is therefore 101.5MtCO2eq. Which makes about 12tCO2eq/capita that year.